In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
import numpy as np

In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

batch_size = 8
num_epochs = 50
learning_rate = 0.0011
data_size = [100, 300, 600, 900, 1200, 1500, 2300]

normal_dir = 'C:\\Users\\WorkStation\\Desktop\\캡디이미지\\CAE_dataset\\normal'
abnormal_dir = 'C:\\Users\\WorkStation\\Desktop\\캡디이미지\\CAE_dataset\\abnormal'

In [3]:
normal_images = datasets.ImageFolder(
    normal_dir,
    transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])
)

abnormal_images = datasets.ImageFolder(
    abnormal_dir,
    transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])
)
    

labeled_abnormal_images = []
for feature, index in abnormal_images:
    labeled_abnormal_images.append([feature, 1])

In [4]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5) 
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 12, 5) 
        self.conv3 = nn.Conv2d(12, 24, 4)  
        self.fc1 = nn.Linear(24 * 25 * 25, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x))) # 6 * 110 * 110
        x = self.pool(F.relu(self.conv2(x))) # 12 * 53 * 53
        x = self.pool(F.relu(self.conv3(x))) # 24 * 25 * 25
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [5]:
for number, dsize in enumerate(data_size, 0):
    
    CNN_PATH = f'CNN_model/CNN_best_model_{number}.pth'
    
    print(f"Number of abnormal data : {dsize}")
    
    train_loader = torch.utils.data.DataLoader(normal_images + labeled_abnormal_images[:dsize], batch_size=batch_size)
    test_loader = torch.utils.data.DataLoader(normal_images + labeled_abnormal_images[:dsize], batch_size=batch_size, shuffle = True)
    
    cnn = CNN()
    cnn.to(device)
    
    optimizer = torch.optim.SGD(cnn.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(train_loader, 0):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = cnn(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        
        if epoch % 10 == 0:
            print(f"[{epoch}/{num_epochs}] loss : {running_loss / len(train_loader)}")
    
    torch.save(cnn.state_dict(), CNN_PATH)
    print("save model " + CNN_PATH ,"\n")

Number of abnormal data : 100
[0/50] loss : 0.7250301480293274
[10/50] loss : 0.6187043702602386
[20/50] loss : 0.5767399391531944
[30/50] loss : 0.5763492518663407
[40/50] loss : 0.5759631159901619
save model CNN_best_model_0.pth 

Number of abnormal data : 300
[0/50] loss : 0.6923602135976156
[10/50] loss : 0.692968369325002
[20/50] loss : 0.6928785689671835
[30/50] loss : 0.6927122036616008
[40/50] loss : 0.6924572745958965
save model CNN_best_model_1.pth 

Number of abnormal data : 600
[0/50] loss : 0.6844152217464787
[10/50] loss : 0.610640885441431
[20/50] loss : 0.435190885494064
[30/50] loss : 0.2526887259612392
[40/50] loss : 0.06587739818600571
save model CNN_best_model_2.pth 

Number of abnormal data : 900
[0/50] loss : 0.7075090209643046
[10/50] loss : 0.5007971621553103
[20/50] loss : 0.45376778083542985
[30/50] loss : 0.3327189119470616
[40/50] loss : 0.2835446327909206
save model CNN_best_model_3.pth 

Number of abnormal data : 1200
[0/50] loss : 0.685674040075292
[10/50